# Workshop Experiments: Supplementary Information

This Jupyter Notebook provides some additional information about the
Kibble-Zurek and Landau-Zener experiments demonstrated in the workshop, as well
as some related notes.

## Learning Resources

Learn more from the following sources:

*   [Coherent quantum annealing in a programmable 2,000 qubit Ising chain, Nature Physics (2022)]
    (https://doi.org/10.1038/s41567-022-01741-6)

    Figure 2A reproduced from [preprint](https://doi.org/10.48550/arXiv.2202.05847)

*   [Quantum critical dynamics in a 5,000-qubit programmable spin glass, Nature (2023)]
    (https://doi.org/10.1038/s41586-023-05867-2)

    Figure 2C reproduced from [preprint](https://doi.org/10.48550/arXiv.2207.13800)

*   [Additional publications](https://www.dwavequantum.com/learn/publications/)

    Publications page on D-Wave's corporate website

*   [D-Wave's YouTube channel](https://www.youtube.com/@dwavequantum)

*   [Per-QPU Solver Properties](https://docs.dwavequantum.com/en/latest/quantum_research/solver_properties_specific.html)

    D-Wave documentation site's page for information on a particular QPU's
    schedule, energy scales, single-qubit freeze out point, etc

*   [D-Wave Documentation Site](https://docs.dwavequantum.com/en/latest/index.html)

*   [D-Wave's GitHub](https://github.com/dwavesystems/)

    Repository for Ocean software and additional software

*   [dwave-examples GitHub Repository](https://cloud.dwavesys.com/leap/examples)

    D-Wave's Ocean code examples

*   [Leap Resources](https://cloud.dwavesys.com/leap/resources)

    Learning resources page in the Leap service

## Embedding

Ocean software's [minorminer](https://docs.dwavequantum.com/en/latest/ocean/api_ref_minorminer/source/index.html)
has a variety of heuristic and guaranteed-performance tools.

*   General mappings: [minorminer.find_embeddings](https://docs.dwavequantum.com/en/latest/ocean/api_ref_system/generated/minorminer.find_embedding.html#minorminer-find-embedding)

*   1:1 mappings (native/subgraph): **[minorminer.subgraph.find_subgraph]()**

*   Clique and biclique embedding: [minorminer.busclique.find_clique_embedding](https://docs.dwavequantum.com/en/latest/ocean/api_ref_minorminer/source/clique_embedding.html#minorminer.busclique.find_clique_embedding)

Choose the right tool: a bad choice can be very inefficient. Consider a customized
approach for regular cases; e.g.:

*   Defect-free 1:1 (next-neighbor-square, Kings, square, triangle) or
    defect-free chains (cubic):
    [hybrid.decomposers.make_origin_embeddings](https://docs.dwavequantum.com/en/latest/ocean/api_ref_hybrid/generated/hybrid.decomposers.make_origin_embeddings.html#hybrid.decomposers.make_origin_embeddings)

*   Parallelization of embeddings (support recently added to Ocean):**[minorminer.utils.parallel_embedding.find_multiple_embeddings]()**

The code cell below finds an embedding for a frustrated loop of length 3:

In [ ]:
# An embedding of a frustrated loop of length 3

from minorminer.subgraph import find_subgraph
from dwave.system import DWaveSampler

qpu = DWaveSampler()

h = {i: 0 for i in range(3)}
J = {(0,1): 1, (1,2): 1, (0,2): 1}  # An antiferromagnetic 3-cycle

emb1to1 = find_subgraph(J.keys(), qpu.to_networkx_graph())  # Edge list
print(emb1to1)

emb = {k: (v,) for k, v in emb1to1.items()}

## Noise-Mitigation/Investigation

Consider the following noise-mitigation strategies:

### Shimming

Most D-Wave research papers use shimming. General-purpose calibration can be
refined for your particular model to eliminate specific cross-talks and
non-idealities.

*   [Calibration-Refinement Paper](https://doi.org/10.3389/fcomp.2023.1238988)

    This published paper also points to recently-updated demonstration code.
    You can tune parameters such as flux biases to restore expected symmetries.

*   [Pytorch Plug-in Code](https://github.com/dwavesystems/dwave-pytorch-plugin/blob/main/README.rst)

    This plug-in can be extended in principle to learning general distributions.

### Spin-Reversal Transforms

You can also use [spin-reversal transforms](https://docs.dwavequantum.com/en/latest/quantum_research/solver_configuration.html#spin-reversal-gauge-transforms) (SRT).

*   SRT with respect to the noise-free evolution amounts to a change of basis (gauge)

*   SRT respect to the programmed realization is a change of basis that leads to
    new noise patterns.

By averaging you eliminate noise: e.g., the effect of independent and identically
distributed flux (h) noise is eliminated at leading order.

```
    qpu = DWaveSampler(solver='Advantage2_system1.1')
    sampler = SpinReversalTransformComposite(qpu)  # Composites applies (and inverts) the transformation
```

### Averaging over Embeddings

Another option is averaging over embeddings.

*   Use randomized or parallelized embeddings:

```
    import networkx as nx

    qpu = DWaveSampler(solver='Advantage2_system1.1')
    sampler1 = SpinReversalTransformComposite(qpu)
    source = nx.from_edgelist([(i, i+1) for i in range(3)])  # A path (1D chain) of size 4
    sampler2 = ParallelEmbeddingComposite(sampler2)  # Composites distributes and inverts (a distribution on 4-spins is returned)
```

*   Exploit symmetries of the source model (Ising model) to permute a known embedding.

*   Many physics models have high symmetry: translational, rotational and
    reflection symmetry. These might be put in manually or a tool like ``pynauty``
    can be used (see the shimming tutorial referenced above).

Clique, biclique and lattice source models have many symmetries. You can permute
keys to find new valid embeddings.The code cell below exploits these symmetries
to vary embeddings:

In [ ]:
# Exploit symmetries to vary embeddings for clique, biclique and lattice models

from dwave.embedding import is_valid_embedding

translation_on_3cycle = {0:1, 1:2, 2:0}
emb2 = {translation_on_3cycle[k]: v for k,v in emb.items()}
print('Original valid embedding', emb)

assert is_valid_embedding(emb, J.keys(), qpu.to_networkx_graph())

print('New valid embedding exploiting symmetries of the source', emb2)

assert is_valid_embedding(emb2, J.keys(), qpu.to_networkx_graph())

Use of spin-reversal transforms to average programmings related by symmetry can
mitigate for flux noise. For the frustrated loop generated above, this increases
the uniformity across its six ground states. Excited states are not typically observed.

The code cell below samples with SRTS and prints out the resulting samples.

In [ ]:
# Use of spin-reversal transforms to average programmings

from dwave.preprocessing import SpinReversalTransformComposite
from dwave.system.composites import FixedEmbeddingComposite

sampler = FixedEmbeddingComposite(SpinReversalTransformComposite(qpu), embedding=emb)

ss = sampler.sample_ising(h, J, num_reads=1000)

print(ss.record.energy, ss.record.num_occurrences)

## Schedules and QPU-Specific Properties

Find the QPU schedule and properties on the [D-Wave documentation website](https://docs.dwavequantum.com/en/latest/quantum_research/solver_properties_specific.html).

The following code cells plot the default and a customized anneal schedule,
assuming the Hamiltonian $H = -A(s)/2 \sum_i X_i + B(s)/2 H_p$.


In [ ]:
# Plot the default energy scales

import numpy as np
import matplotlib.pyplot as plt
from dwave.system import energy_scales_custom_schedule

sABc = np.loadtxt('helper_files/09-1312A-A_Advantage2_system1.1_fast_annealing_schedule.csv', delimiter=",", skiprows=1)

plt.plot(sABc[:,0], sABc[:,1])
plt.plot(sABc[:,0], sABc[:,2])
plt.ylabel('Energy scale, GHz')
plt.xlabel('Normalized anneal time, t/t_a')

In [ ]:
# Plot customized energy scales

anneal_schedule = [[0.0, 1.0], [5, 0.45], [99, 0.45], [100, 1.0]]

tsABc = energy_scales_custom_schedule(sABc, custom_schedule =anneal_schedule)

plt.plot(tsABc[:,0], tsABc[:,1])
plt.plot(tsABc[:,0], tsABc[:,2])
plt.ylabel('Energy scale, GHz')
plt.xlabel('Normalized anneal time, t/t_a')

## Temperature (quasi-static freezeout)

If the model is maximum entropy subject to some rate of excitations (energy), it is a Boltzmann distribution.

This is expected for single-late freeze-out (thermal equilibrium up to the small A(s) limit, then dynamical arrest)

The local pattern of excitations is sufficient to efficiently infer the temperature with enough samples:
```
    qpu = DWaveSampler(solver='Advantage2_system1.1')
    dwave.system.temperatures.fast_effective_temperature(qpu)  # By default, uses independent spins
```
Note that, if the distribution is not Boltzmann due to ergodicity breaking (trapping by local minima) we expect the rate of local excitations to be smaller than the rate of general excitations. Quantum-Boltzmann and noise effects can also lead to bias in estimators.

In [ ]:
# Calculate the maximum-likelihood estimate for T, where P(s) = exp(- 1/T sum_i h_i) and h_i ~ [-1/4, 1/4] independent and identically distributed
from dwave.system.temperatures import fast_effective_temperature
from dwave.system import DWaveSampler
for solver in ['Advantage_system4.1', 'Advantage2_prototype2.6', 'Advantage2_system1.1']:
    qpu = DWaveSampler(solver=solver)
    T, _ = fast_effective_temperature(qpu, h_range=(-1/4, 1/4))  # By default, uses independent spins
    print(f'Unitless temperatures characterizing {solver}', T)

## Classical dynamics

We include a variety of heuristic optimizer and classical dynamics include path-integral monte carlo, rotor models and simulated annealing

1. Full randomized Gibbs sampling at fixed temperature
2. Continuous-time path integral Monte-Carlo

In [ ]:
from dwave.samplers import SimulatedAnnealingSampler, PathIntegralAnnealingSampler

h = {i: 0 for i in qpu.nodelist}
J = {e: np.random.random() for e in qpu.edgelist}  # Spin glass

num_reads = 32  # Small sampleset for the demo

# Fixed temperature classical sampler
sampler = SimulatedAnnealingSampler()  # Fixed temperature (this example) classical dynamics
ss = sampler.sample_ising(h, J, beta_schedule=[1/T], num_sweeps_per_beta = 16, num_reads=num_reads, beta_schedule_type='custom', randomize_order=True, acceptance_criteria='Gibbs')
plt.plot(np.sort(ss.record.energy), np.arange(num_reads)/num_reads, label='Gibbs at T')

# Path integral sampling
s_freezeout = 0.684
s_interps = np.arange(16)/15*s_freezeout
Hp_field = np.interp(s_interps, sABc[:,0], sABc[:,2])/T  # Dynamics stop around 0.684, single qubit freezeout
Hd_field = np.interp(s_interps, sABc[:,0], sABc[:,1])/T
sampler = PathIntegralAnnealingSampler()  # Simulated Quantum Annealing
ss = sampler.sample_ising(h, J, Hd_field=Hd_field,Hp_field=Hp_field, num_reads=num_reads, beta_schedule_type='custom')

plt.plot(np.sort(ss.record.energy), np.arange(num_reads)/num_reads, label='Path Integral at T given schedule')
plt.ylabel('Cumulative Distribution Function')
plt.xlabel('Energy of high precision spin-glass')
plt.legend()

# Timing

Timing information is returned as part of the sample set.

We can also estimate the access-time using tools prior to submission

In [ ]:
print('Estimated qpu access time', qpu.solver.estimate_qpu_access_time(num_qubits=len(h), num_reads=num_reads))
num_reads = 1000
ss = qpu.sample_ising(h, J, num_reads=num_reads, answer_mode='raw')
print(ss.info['timing'])